In [1]:
import numpy as np

# Explanation

(1)hidden_size: output dimension 

-> result of hidden_state: h_t.shape = (hidden_size, 1)

(2)vstack: stack in row direction

 -> rsult of vstack: concat_vector.shape = (input_size + hidden_size , 1)

* but it can only same size col.





In [2]:
class activation_function:
    def __init__(self):
        pass

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def tanh(self, x):
        return np.tanh(x)

class LSTM(activation_function):
    def __init__(self, hidden_size, concat_size):
        # z  = alpha(w @ [h, x] + b) 
        # alpha = activation function
        self.w_f = np.random.randn(hidden_size, concat_size)
        self.b_f = np.zeros((hidden_size, 1))
        self.w_o = np.random.randn(hidden_size, concat_size)
        self.b_o = np.zeros((hidden_size, 1))
        self.w_c = np.random.randn(hidden_size, concat_size)
        self.b_c = np.zeros((hidden_size, 1))
        self.w_i = np.random.randn(hidden_size, concat_size)
        self.b_i = np.zeros((hidden_size, 1))
    
    def forget_gate(self, concat_vector):
        value = (self.w_f @ concat_vector) + self.b_f
        f_t = self.sigmoid(value)
        return f_t
    
    def input_gate(self, concat_vector):
        value = (self.w_i @ concat_vector) + self.b_i
        i_t = self.sigmoid(value)
        return i_t
    
    def candidate_memory(self, concat_vector):
        value = (self.w_c @ concat_vector) + self.b_c
        c_t_hat = np.tanh(value)
        return c_t_hat
    
    def output_gate(self ,concat_vector):
        value = (self.w_o @ concat_vector) + self.b_o
        o_t = self.sigmoid(value)
        return o_t

    def cell_state_update(self, c_prev ,concat_vector):
        c_t_hat = self.candidate_memory(concat_vector)
        f_t = self.forget_gate(concat_vector)
        i_t = self.input_gate(concat_vector)
        c_t = (f_t * c_prev) + (i_t * c_t_hat)
        return c_t
    
    #print summarized information
    def hidden_state(self, c_t, concat_vector):
        h_t = self.output_gate(concat_vector) * np.tanh(c_t)
        return h_t
    
    def forward(self, h_prev, x_t , c_prev):
        concat_vector = np.vstack((h_prev, x_t))
        c_t = self.cell_state_update(c_prev, concat_vector)
        h_t = self.hidden_state(c_t, concat_vector)
        return h_t, c_t
        

# example_data_set

(1)lstm only get 3_dimension data_set
    
    -> we need to data process

(2) because data is just an array of numbers,

* we need to create a rule to input into lstm.

    def create_sequences divides an array of numbers into time_steps,

    and creates multiple samples.

In [3]:
train_set = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
test_set = [1, 2, 3, 4, 5, 6]
train_set = np.array(train_set)
test_set = np.array(test_set)

def create_sequences(data, time_steps =3):
    x, y = [], []
    for i in range(len(data) - time_steps): #10 - 3 = 7
        seq_x = data[i:i + time_steps]
        #0~6, 1~7, 2~8, 3~9, 4~10 (sample)
        seq_y = data[i + time_steps]
        #7, 8, 9, 10 (correct label)
        x.append(seq_x)
        y.append(seq_y)
    return np.array(x), np.array(y)

#==================make matrix==================
train_data_x , train_data_y = create_sequences(train_set)
#==================check data shape=============
print(train_data_x.shape, train_data_y.shape)

#==================reshape 3_dimension==========
train_data_x = train_data_x.reshape((train_data_x.shape[0], train_data_x.shape[1], 1))
print(train_data_x.shape)

(7, 3) (7,)
(7, 3, 1)


# ex_1 basic lstm

- this result is not accurate. 

    so we need backward definition.

In [5]:
input_size = 1
hidden_size = 4
lstm = LSTM(input_size, concat_size = input_size + hidden_size)

h = np.zeros((hidden_size, 1))
c = np.zeros((hidden_size, 1))

last_seq = test_set[-3: ]
for i in range(100):
    for i in last_seq:
        x_t = np.array([i])
        h, c = lstm.forward(h, x_t, c)

next_value = h.mean()
print(next_value)



0.11757742987146913


In [ ]:
class Dense:
    def __init__(self, input_size):
        self.w = np.random.randn(1, input_size) * 0.01
        self.b = np.zeros((1,1))
        self.lr = 0.01
    
    def forward(self, h):
        self.h = h
        y_pred = self.w @ h + self.b
        return y_pred
    
    def backward(self, dy):
        self.dw = dy @ self.h.T
        self.db = dy
        self.dh = self.w.T @ dy
        return self.dh
    
    def update(self):
        self.w -= self.lr * self.dw
        self.b -= self.lr * self.db

class calculator:
    def __init__(self):
        pass
    
    def mse_loss(self, y_pred, y_true):
        return np.mean((y_pred - y_true) ** 2)
    
    def mse_grad(self, y_pred, y_true):
        return 2 * (y_pred - y_true)
    
    